# Stone Tech Challenge

## Libraries e modules imports

In [1]:
import duckdb
import os
from datetime import date
from dotenv import load_dotenv
from google.cloud import storage

## Configuration

In [16]:
load_dotenv('./postgres/.env')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), 'hop/ENV/key.json')
bucket_name = 'desafio-eng-dados'
prefix = str(date.today().year)

## Data extraction from GCS bucket

In [19]:


def download_blobs(*kwargs):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)

    for blob in blobs:
        blob_name = blob.name
        local_blob_path = './blobs/'
        blob_path = os.path.join(local_blob_path, blob_name)
        blob.download_to_filename(blob_path)


download_blobs()

RefreshError: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})

## Creating OLTP database with PostgreSQL and DuckDB

### DuckDB connection

In [18]:
conn = duckdb.connect()

user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_HOST')
port = os.getenv('POSTGRES_PORT')
dbname = os.getenv('POSTGRES_DB')

conn.execute(f"ATTACH 'postgresql://{user}:{password}@{host}:{port}/{dbname}' AS db (TYPE POSTGRES);")

### Creating temporary table

In [11]:
conn.execute("""
CREATE OR REPLACE TEMP TABLE original AS
    SELECT * FROM read_parquet('blobs/2024*')
""")

### orders table

In [16]:
conn.execute("""
INSERT INTO db.public.orders (
    order_number, terminal_id, customer_id, cancellation_reason, last_modified_date, arrival_date, deadline_date
)
SELECT
    order_number, terminal_id, customer_id, cancellation_reason, last_modified_date, arrival_date, deadline_date
FROM original
""")

### terminals table

In [15]:
conn.execute("""   
INSERT INTO db.public.terminals (
    terminal_id, provider_id, technician_id, terminal_serial_number, terminal_model, terminal_type
)
SELECT
    distinct(terminal_id),
    provider_id,
    technician_id,
    terminal_serial_number, 
    terminal_model, 
    terminal_type, 
FROM original
    JOIN db.public.providers
        ON original.provider = providers.provider
    JOIN db.public.technicians
        ON original.technician_email = technicians.technician_email;
""")

### customers table

In [14]:
conn.execute("""
INSERT INTO db.public.customers (
    customer_id, customer_phone, city, country, country_state, zip_code, street_name, complement, neighborhood
)
SELECT
    distinct(customer_id), customer_phone, city, country, country_state, zip_code, street_name, complement, neighborhood
FROM original
""")

### providers table

In [13]:
conn.execute("""
INSERT INTO db.public.providers (
    provider
)
SELECT
    distinct(provider)
FROM original
""")

### technicians table

In [12]:
conn.execute("""
INSERT INTO db.public.technicians (
    technician_email
)
SELECT
    distinct(technician_email)
FROM original
""")

In [ ]:
conn.execute("""
INSERT INTO db.public.tabelao (
    customer_id, customer_phone, city, country, country_state, zip_code, street_name, complement, neighborhood,
             terminal_id, terminal_serial_number, terminal_model, terminal_type, order_number, 
             cancellation_reason, last_modified_date, arrival_date, deadline_date, provider, technician_email
)
SELECT
    customer_id, customer_phone, city, country, country_state, zip_code, street_name, complement, neighborhood,
             terminal_id, terminal_serial_number, terminal_model, terminal_type, order_number, 
             cancellation_reason, last_modified_date, arrival_date, deadline_date, provider, technician_email
             FROM read_parquet('blobs/2024*')
""")

In [ ]:
duckdb.sql("""select terminal_id from read_parquet('blobs/2024*');""")